In [2]:
#coding:utf-8
import networkx as nx
from networkx.algorithms import bipartite
import pandas as pd
import numpy as np
import re
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import os
import cn2an
import chinese2digits as c2d
import operator
import plotly.express as px
import plotly
import plotly.graph_objects as go
import plotly as py
import plotly.offline as offline
offline.init_notebook_mode(connected=True)
from urllib.request import urlopen
import json
import requests
from plotly.subplots import make_subplots
from collections import Counter
mapbox_access_token = "pk.eyJ1Ijoic3RhcmljZSIsImEiOiJjazN6Y2s5dTUxY2R6M2xxcHllbXk4YWFzIn0.lActFqLzqRWGn7dqr4BShw"
px.set_mapbox_access_token(mapbox_access_token)

In [4]:
base_url = "/Users/starice/Desktop/total_extracted_result/"
pre_dir = ['type1', 'type2', 'type3', 'type4']
dir_name = ['2014', '2015', '2016', '2017', '2018', '2019', '2020']
dir_sname = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12']
# csv_graph, _new_csvpd = convert_csv_graph(pre_dir[:], dir_name[:], dir_sname[:])
# csv_graph = nx.read_gpickle("/Users/starice/Desktop/csv_graph.gpickle")

In [3]:
# 在后面绘制地区区域划分图会用到
# with open("/Users/starice/OwnFiles/cityu/RA/code/case_process/china_province.geojson") as f:
#     provinces_map = json.load(f)
mapbox_access_token = "pk.eyJ1Ijoic3RhcmljZSIsImEiOiJjazN6Y2s5dTUxY2R6M2xxcHllbXk4YWFzIn0.lActFqLzqRWGn7dqr4BShw"
px.set_mapbox_access_token(mapbox_access_token)

In [5]:
all_cases = pd.read_csv('/Users/starice/OwnFiles/cityu/RA/case_study/data/total_extracted_result/all_cases.csv', encoding="utf-8")
#获取所有一审案件
first_cases = all_cases[all_cases['procedure']=="一审"]

In [15]:
degree_1stdfs = all_cases.groupby("defendant")['case_id'].unique().reset_index()
degree_1stdfs['case_count'] = degree_1stdfs['case_id'].apply(lambda r: len(r))
degree_1stdfs.sort_values(by="case_count", inplace=True, ascending=False)
fds_200 = degree_1stdfs[:200]
fds_200.head()

,defendant,case_id,case_count
4735,天津市人人乐商业有限公司,"[57aa73eec2265c28a555d425, 581043732f12c631927...",2398
14160,重庆永辉超市有限公司,"[581136be2f12c63192b38df6, 57a951bfc2265c28a54...",1090
13970,重庆华润万家生活超市有限公司,"[57ad2054c2265c04d1192e9a, 581036862f12c631927...",481
9974,浙江淘宝网络有限公司,"[57ab9058c2265c28a560195d, 57a9f83ac2265c04d1f...",481
4775,天津市河东区宽巷子小吃店,"[5da6d68ca24985727533cb51, 5df563a6ab936e46149...",371


In [14]:
#获取所有一审案件
first_cases = all_cases[all_cases['procedure']=="一审"]
print("一审案件数量： ", len(first_cases['case_id'].drop_duplicates()))

# 获取所有案件原告的节点度并排序
degree_1stdfs = first_cases.groupby("defendant")['case_id'].unique().reset_index()
degree_1stdfs['case_count'] = degree_1stdfs['case_id'].apply(lambda r: len(r))
degree_1stdfs.sort_values(by="case_count", inplace=True, ascending=False)
fds_200 = degree_1stdfs[:200]
fds_200.head()

一审案件数量：  32311


,defendant,case_id,case_count
4073,天津市人人乐商业有限公司,"[57aa73eec2265c28a555d425, 581161a82f12c63192b...",2386
11910,重庆永辉超市有限公司,"[581136be2f12c63192b38df6, 57a951bfc2265c28a54...",1087
11729,重庆华润万家生活超市有限公司,"[57ad2054c2265c04d1192e9a, 581212002f12c63192c...",473
8490,浙江淘宝网络有限公司,"[57ab9058c2265c28a560195d, 57ab9686c2265c28a56...",400
4112,天津市河东区宽巷子小吃店,"[5df563a6ab936e4614986c02, 5ee076333edbe71b2f8...",370


In [9]:
fds_10 = degree_1stdfs[:10]
new_selected_1stcp = first_cases[first_cases['defendant'].isin(fds_10['defendant'])]

In [10]:
degree_1stdfs[:30]
# 看一下被告到至少100次的被告是否有案件数量逐年下降的趋势

,defendant,case_id,case_count
4073,天津市人人乐商业有限公司,"[57aa73eec2265c28a555d425, 581161a82f12c63192b...",2386
11910,重庆永辉超市有限公司,"[581136be2f12c63192b38df6, 57a951bfc2265c28a54...",1087
11729,重庆华润万家生活超市有限公司,"[57ad2054c2265c04d1192e9a, 581212002f12c63192c...",473
8490,浙江淘宝网络有限公司,"[57ab9058c2265c28a560195d, 57ab9686c2265c28a56...",400
4112,天津市河东区宽巷子小吃店,"[5df563a6ab936e4614986c02, 5ee076333edbe71b2f8...",370
6286,成都蓉李记餐饮管理有限公司,"[5df563a6ab936e4614986c02, 5ee076333edbe71b2f8...",370
7988,沈阳家乐福商业有限公司,"[57aab194c2265c04d1012292, 581069ab2f12c631928...",320
3741,四川徽记食品股份有限公司,"[5a1700aee13823d7213e28a9, 5a170129e13823d7213...",308
4085,天津市人人乐商业有限公司滨海新区河北路购物广场,"[57b05c8ac2265c7bd4edbd63, 57b05c8cc2265c7bd4e...",290
11909,重庆永辉云创科技有限公司,"[5ec808177116c36f1b9836b2, 5e2829b7a163f656afc...",283


In [11]:
time_nselected_1stcp = new_selected_1stcp.groupby(['defendant', 'year'])['case_id'].nunique().reset_index()
time_nselected_1stcp.rename(columns={"case_id": "case_count"}, inplace=True)

In [12]:
fig = px.box(
    time_nselected_1stcp, 
    x = "year", 
    y = "case_count", 
#     color = "defendant"
)
fig.update_yaxes(type="log")
fig.show()

In [13]:
fig = px.line(
    time_nselected_1stcp, 
    x = "year", 
    y = "case_count", 
    color = "defendant"
)
# fig.update_yaxes(type="log")
fig.show()